# 建图

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import math

In [ ]:
with open('MC1.json', 'r') as file:
    json_data = json.load(file)
nodes = json_data['nodes']
links = json_data['links']

In [ ]:
G = nx.DiGraph()
edges = []
for link in links:
    edges.append((link['source'], link['target']))
edges = list(set(edges))
G.add_edges_from(edges)

In [ ]:
suspects  = ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]

# 在指定长度内统计环的数量

In [ ]:
def dfs_paths(graph, start_node, current_node, steps_left, visited, path, all_paths, suspect):
    suspect_copy = suspect.copy()
    if (current_node in ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]) and (current_node not in suspect):
        suspect_copy.append(current_node)

    if steps_left == 0 and current_node == start_node:
        all_paths[len(suspect_copy)][0].append(path + [current_node])  # 找到了符合条件的路径
        if len(suspect_copy) == 1:
            if suspect_copy[0] == "Mar de la Vida OJSC":
                all_paths[1][1][0]+=1
            elif suspect_copy[0] == "979893388":
                all_paths[1][1][1]+=1
            elif suspect_copy[0] == "8327":
                all_paths[1][1][2]+=1
        elif len(suspect_copy) == 2:
            if (suspect_copy[0] == "Mar de la Vida OJSC" and suspect_copy[1] ==  "979893388") or (suspect_copy[1] == "Mar de la Vida OJSC" and suspect_copy[0] ==  "979893388"):
                all_paths[2][1][0]+=1
            elif (suspect_copy[0] == "Mar de la Vida OJSC" and suspect_copy[1] ==  "8327") or (suspect_copy[1] == "Mar de la Vida OJSC" and suspect_copy[0] ==  "8327"):
                all_paths[2][1][1]+=1
            elif (suspect_copy[0] == "979893388" and suspect_copy[1] == "8327") or (suspect_copy[1] == "979893388" and suspect_copy[0] ==  "8327"):
                all_paths[2][1][2]+=1
        else:  # 0或者3
            all_paths[len(suspect_copy)][1][0]+=1

    if current_node in visited or steps_left < 0:
        return  # 终止递归

    visited.add(current_node)
    next_nodes = graph[current_node]
    for next_node in next_nodes:
        dfs_paths(graph, start_node, next_node, steps_left - 1, visited, path + [current_node], all_paths, suspect_copy)
    visited.remove(current_node)


def find_all_cycles(graph, start_node, num_steps):
    visited = set()
    all_paths = [[[],[0]], [[],[0,0,0]], [[],[0,0,0]], [[],[0]]]   #存0个，1个(M, 9, 8)，2个(M,+9 M+8, 9+8)，3个,（4个无）
    dfs_paths(graph, start_node, start_node, num_steps, visited, [], all_paths, [])
    return all_paths

In [ ]:
def count_cycles(G, node, start_length, end_length):
    X = np.arange(start_length, end_length+1)
    count = [[], [], [], []]
    record = []
    for x in X:
        all_paths = find_all_cycles(G, node, x)  # [[], [], [], []]
        paths = []
        for i in range(len(all_paths)):
            paths.append(all_paths[i][0])
        record.append(paths)

        for i in range(len(paths)):
            count[i].append(all_paths[i][1])

    print("Details of count:")
    for i in range(len(count)):
        print(count[i])

    return count

In [ ]:
def count_plot(count_detail, node, start_length, end_length):
    X = np.arange(start_length, end_length+1)

    width = 0.3

    count = [[],[],[],[]]
    for i in range(len(count_detail)):
        for j in range(len(count_detail[i])):
           count[i].append(np.sum(count_detail[i][j]))

    print("----------------------------\nCount:")
    for c in count:
        print(c)
    print("----------------------------")

    p1 = plt.bar(X, count[0], width)#, yerr=menStd)

    p2 = plt.bar(X, count[1], width, bottom=count[0])#, yerr=womenStd)

    temp = [count[0],[],[],[]]
    for i in range(0,len(count[0])):
        sum = count[0][i] + count[1][i]
        temp[1].append(sum)

    p3 = plt.bar(X, count[2], width, bottom=temp[1])

    for i in range(0,len(count[0])):
        sum = temp[1][i] + count[2][i]
        temp[2].append(sum)

    p4 = plt.bar(X, count[3], width, bottom=temp[2])

    for i in range(0,len(count[0])):
        sum = temp[2][i] + count[3][i]
        temp[3].append(sum)

    a = max(temp[3])

    plt.ylabel('Length')
    plt.ylabel('Count')
    plt.title(f'Counts of cycles of lenth {start_length} to {end_length} ({node})')
    plt.xticks(X)
    plt.yticks([0, math.ceil(1.1*a)])
    plt.legend((p1[0], p2[0], p3[0], p4[0]), ('0 suspect', '1 suspect', '2 suspect', '3 suspect'))

    # 标注数据
    for i in range(len(X)):
        for j in range(len(count)):
            x = X[i]
            y = count[j][i]
            if y!=0:
                if y == 1:
                    plt.text(x, temp[j][i]-0.75, str(y)+"\n"+str(count_detail[j][i]), ha='center', va='bottom')
                else:
                    plt.text(x, temp[j][i]-2, str(y)+"\n"+str(count_detail[j][i]), ha='center', va='bottom')

    # 显示图形
    plt.show()

In [ ]:
def detail_plot(count_detail, node, start_length, end_length):
    labels_1 = ["Mar de la Vida OJSC", "979893388", "8327"]
    labels_2 = ["Mar de la Vida OJSC and 979893388", "Mar de la Vida OJSC and 8327", "979893388 and 8327"]

    for i in range(end_length-start_length+1):
        if node not in labels_1:
            if np.sum(count_detail[1][i])!=0:
                plt.pie(count_detail[1][i], labels=labels_1, autopct='%1.1f%%', shadow=True)
                plt.title(f"1 suspected entries [length {i+start_length}]({node})")
                plt.show()
            if np.sum(count_detail[2][i])!=0:
                # 在第二个子图上绘制饼状图
                plt.pie(count_detail[2][i], labels=labels_2, autopct='%1.1f%%', startangle=90, shadow=True)
                plt.title(f"2 suspected entries [length {i+start_length}]({node})")
                plt.show()
        else:
            if np.sum(count_detail[2][i])!=0:
                plt.pie(count_detail[2][i], labels=labels_2, autopct='%1.1f%%', startangle=90, shadow=True)
                plt.title(f"2 suspected entries [length {i+start_length}]({node})")
                plt.show()

        # 显示图表
        # plt.show()

In [ ]:
count0 = count_cycles(G, suspects[0], start_length=2, end_length=7)

In [ ]:
count_plot(count0, suspects[0], start_length=2, end_length=7)

In [ ]:
detail_plot(count0, suspects[0],start_length=2, end_length=7)

### 测试运行时间

In [ ]:
import time

start_time = time.time()

paths = find_all_cycles(G, suspects[0], 8)
print(f"There are {len(paths)} cycles of length 8 ({suspects[0]})")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"程序运行时间：{elapsed_time}秒")

In [ ]:
count1 = count_cycles(G, suspects[0], start_length=2, end_length=7)
count_plot(count1, suspects[0], start_length=2, end_length=7)

In [ ]:
detail_plot(count1, suspects[0],start_length=2, end_length=7)

In [ ]:
count2 = count_cycles(G, suspects[1], start_length=2, end_length=7)
count_plot(count2, suspects[1], start_length=2, end_length=7)

In [ ]:
detail_plot(count2, suspects[1],start_length=2, end_length=7)

In [ ]:
count3 = count_cycles(G, suspects[2], start_length=2, end_length=7)
count_plot(count3, suspects[2], start_length=2, end_length=7)

In [ ]:
count4 = count_cycles(G, suspects[3], start_length=2, end_length=7)
count_plot(count4, suspects[3], start_length=2, end_length=7)

In [ ]:
detail_plot(count4, suspects[3],start_length=2, end_length=7)

### Others

In [ ]:
count5 = count_cycles(G, '7775', start_length=2, end_length=7)
count_plot(count5, '7775', start_length=2, end_length=7)

In [ ]:
detail_plot(count5, '7775', start_length=2, end_length=7)

In [ ]:
count6 = count_cycles(G, 'png xi  Line', start_length=2, end_length=5)
count_plot(count6, 'png xi  Line', start_length=2, end_length=5)

In [ ]:
detail_plot(count6, 'png xi  Line', start_length=2, end_length=5)

In [ ]:
count7 = count_cycles(G, '1', start_length=2, end_length=7)
count_plot(count7, '1', start_length=2, end_length=7)

In [ ]:
detail_plot(count7, '1', start_length=2, end_length=7)

# 在指定阶数内统计不同长度环的数量

In [ ]:
def find_cycles(graph, node, order):
    # 筛选指定阶数邻居
    # 将有向图转为无向图
    graph = graph.to_undirected()

    # 获取节点列表
    neighbor_graph = nx.ego_graph(graph, node, radius = order,  undirected=True)
    neighbors = list(neighbor_graph.nodes())

    print(f"There are {len(neighbors)} neighbours. ({order})")

    # 指定阶数内的子图
    G_ego = nx.DiGraph()
    edges_ego = []
    for link in links:
        if link['source'] in neighbors and link['target'] in neighbors:
            edges_ego.append((link['source'], link['target']))
    G_ego.add_edges_from(edges_ego)

    # 指定阶数内的环列表
    cycles = nx.simple_cycles(G_ego)

    selected_cycles = []
    try:
        cycle = next(cycles)
        while cycle:
            if node in cycle:
                selected_cycles.append(cycle)
                #print(cycle)
            cycle = next(cycles, None)

    except StopIteration:
        print("There is no cycle.")

    return selected_cycles

In [ ]:
def histogram_plot(cycles, node, order):
    if len(cycles) != 0:
        # 柱状图的数据
        idx = range(len(cycles))
        lengths = [len(c) for c in cycles]

        # 创建柱状图
        plt.bar(idx, lengths)

        # 添加标题和轴标签
        plt.title(f"Length of cycles in {order}th neighborhoods of {node}")
        plt.xlabel("index")
        plt.ylabel("Length")

        # 标注数据
        for i in range(len(idx)):
            plt.text(i, lengths[i], str(lengths[i]), ha='center', va='bottom')

        # 显示图形
        plt.show()
    else:
        print("None")

def piechart_plot(cycles, node, order):
    if len(cycles) != 0:
        # 饼图数据
        lengths = [len(c) for c in cycles]

        count = Counter(lengths)
        labels = []
        sizes = []
        for value, frequency in count.items():
            labels.append(value)
            #sizes.append(100.0*frequency/len(lengths))
            sizes.append(frequency)

        plt.pie(sizes, labels=labels, autopct='%1.1f%%')
        plt.title(f"Pie Chart of Length Statistics of cycles in {order}th neighborhoods of {node}")
        plt.show()
    else:
        print("None")

### "Mar de la Vida OJSC"

In [ ]:
node = suspects[0]
order = 1
cycles = find_cycles(G, node, order)
print(cycles)

In [ ]:
histogram_plot(cycles, node, order)

In [ ]:
piechart_plot(cycles, node, order)

### "979893388"

In [ ]:
node = suspects[1]
order = 1
cycles = find_cycles(G, node, order)
#print(cycles)

In [ ]:
histogram_plot(cycles, node, order)

In [ ]:
piechart_plot(cycles, node, order)

### "Oceanfront Oasis Inc Carriers"

In [ ]:
node = suspects[2]
order = 1
cycles = find_cycles(G, node, order)
#print(cycles)

In [ ]:
histogram_plot(cycles, node, order)

In [ ]:
piechart_plot(cycles, node, order)

### "8327"

In [ ]:
node = suspects[3]
order = 1
cycles = find_cycles(G, node, order)
#print(cycles)

In [ ]:
histogram_plot(cycles, node, order)

In [ ]:
piechart_plot(cycles, node, order)

### Others

In [ ]:
node = "7775"
order = 1
cycles = find_cycles(G, node, order)
#print(cycles)

In [ ]:
histogram_plot(cycles, node, order)

In [ ]:
piechart_plot(cycles, node, order)

In [ ]:
node = "1"
order = 1
cycles = find_cycles(G, node, order)
#print(cycles)

In [ ]:
histogram_plot(cycles, node, order)

In [ ]:
piechart_plot(cycles, node, order)

In [ ]:
node = "png xi  Line"
order = 1
cycles = find_cycles(G, node, order)
#print(cycles)

In [ ]:
histogram_plot(cycles, node, order)

In [ ]:
piechart_plot(cycles, node, order)

# 环的可视化

In [ ]:
def draw_cycles(G, node, length, include = None):
    paths = find_all_cycles(G, node, length)

    G_cycles = nx.DiGraph()

    node_cycles = []
    node_suspect = []
    edges_cycles = []
    for i in [0,1,2,3]:
        for path in paths[i][0]:
            if include is None:
                node1 = None
                node2 = None
                for n in path:
                    if (n not in node_cycles) and (n not in suspects):
                        node_cycles.append(n)
                    if (n not in node_suspect) and (n in suspects):
                        node_suspect.append(n)
                    if node1 is None:
                        node1 = n
                    else:
                        node2 = node1
                        node1 = n
                        edges_cycles.append((node2,node1))
            else:
                if include in path:
                    node1 = None
                    node2 = None
                    for n in path:
                        if (n not in node_cycles) and (n not in suspects):
                            node_cycles.append(n)
                        if (n not in node_suspect) and (n in suspects):
                            node_suspect.append(n)
                        if node1 is None:
                            node1 = n
                        else:
                            node2 = node1
                            node1 = n
                            edges_cycles.append((node2,node1))

    # 添加边
    G_cycles.add_edges_from(edges_cycles)

    # 确定节点位置布局
    pos = nx.spring_layout(G_cycles)

    # 绘制节点和边
    nx.draw_networkx_nodes(G_cycles, pos, nodelist=node_suspect, node_color='red', node_size=25)
    nx.draw_networkx_nodes(G_cycles, pos, nodelist=node_cycles, node_color='blue', node_size=25)
    nx.draw_networkx_edges(G_cycles, pos, width=0.20, arrowsize=2.5,connectionstyle='arc3,rad=0.2')    #, arrowstyle='->'

    # 添加节点标签
    nx.draw_networkx_labels(G_cycles, pos, font_size=2.5)

    # 显示图形
    if include is None:
        plt.title(f'Cycles of length {length} ({node})')
    else:
        plt.title(f'Cycles of length {length} ({node} and {include})')

    plt.axis('off')
    if node in suspects:
        if include is None:
            plt.savefig(f"{node}\length{length}({node}).pdf")
        else:
            plt.savefig(f"{node}\length{length}({node} and {include}).pdf")
    else:
        if include is None:
            plt.savefig(f"Others\length{length}({node}).pdf")
        else:
            plt.savefig(f"Others\length{length}({node} and {include}).pdf")
    plt.show()

In [ ]:
for i in range(2,7):
    draw_cycles(G, suspects[0], i)

In [ ]:
draw_cycles(G, suspects[0], 5, include='Faroe Islands Shrimp Shark')

In [ ]:
for i in range(2,7):
    draw_cycles(G, 'SeaSpray Wave SRL Solutions', i)